In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import math
import os

#libairies pour le réseau de neurone
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pathlib

In [8]:
os.chdir('D:/ilona/Documents/MASTER/Linux/projet_final/Diamonds/Data')
diamond = pd.read_csv('diamond_final.csv')
diamond.drop('Unnamed: 0', axis=1, inplace=True)
diamond.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,1,2,7,61.5,55.0,326,3.95,3.98,2.43
1,0.21,2,2,6,59.8,61.0,326,3.89,3.84,2.31
2,0.23,4,2,4,56.9,65.0,327,4.05,4.07,2.31
3,0.29,2,6,5,62.4,58.0,334,4.20,4.23,2.63
4,0.31,4,7,7,63.3,58.0,335,4.34,4.35,2.75


In [ ]:
train_dataset = diamond.sample(frac=0.8,random_state=0)
test_dataset = diamond.drop(train_dataset.index)

In [ ]:
#Observation des statistiques globales
train_stats = train_dataset.describe()
train_stats.pop("price")
train_stats = train_stats.transpose()
train_stats

In [ ]:
#Suppression de la variable cible
train_labels = train_dataset.pop('price')
test_labels = test_dataset.pop('price')

In [ ]:
#Standardisation
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
# das neuronale Netz mit seinen Schichten wird gebildet
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse', # optimisation avec l'erreur quadratique moyenne
                optimizer=optimizer,
                metrics=['mae', 'mse']) # évaluation avec les métriques mae et mse
  return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

In [ ]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

#EPOCHS = 300 
EPOCHS = 50
#EPOCHS = 150
#En raison de la taille énorme de notre jeu de données, il n'est pas nécessaire d'avoir beaucoup d'époques pour le calcul
#Pour chaque époque, l'ensemble des données de training est appliqué
history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])
#20% des 80% de données d'entraînement sont utilisés pour l'étape de validation

## Les valeurs MAE de différentes époques ou de leurs dépenses ont été comparées (epoch 50, 150 et 300)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
hist.to_csv('result_NN.csv')
#Données des époques enregistrées dans un nouveau fichier CSV et calcul de la valeur moyenne du MAE sur 10 passages

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [price]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  #plt.ylim([0,5])
  plt.legend()
#Hier wird für jede Epoche der MAE ausgerechnet für Trainings- und Validationdaten
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$price^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  #plt.ylim([0,20])
  plt.legend()
  plt.show()
plot_history(history)

In [ ]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)
print("Testing set Mean Abs Error: {:5.2f} price".format(mae))

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [price]')
plt.ylabel('Predictions [price]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

#x-Achse tatsächliche Werte und y-Achse zeigt unsere Vorhersagen
#Grundsätzlich zeit es, dass die Vorhersagen je teuerer die Diamanten sind günstiger geschätzt werden, als sie in wirklichkeit sind
#Je besser die Diamanten sind, desto seltener sind sie und sind im Verhältnis überproportional teurer
#Beispiel ist ein 20.000 USD teuerer Diamanat, dieser wird teilweise auf 12.000 USD geschätzt, dadurch zeigt sich, dass die Vorhersagen niedriger schätzen als der tatsächliche Wert ist
#die Ausreißer erklären, wäre interessant
#den Graphen anpassen

In [ ]:
error = test_predictions - test_labels
#Der Prozentuale Anteil wird berechnet, der kann leichter interpretiert werden als ein USD
perc_error = ((error/(test_labels))*100)
#in Prozent rechnen, damit man besser visualisieren kann
plt.hist(perc_error, bins = 2500)
plt.xlabel("Prediction Error [price]")
_ = plt.ylabel("Count")